<a href="https://colab.research.google.com/github/blue-create/langlens/blob/main/models/bloom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Imports

In [1]:
%%capture
!pip install transformers==4.20.0

In [2]:
%%capture
!pip install datasets

In [9]:
from tqdm import tqdm
import numpy as np
import pandas as pd
import os
import json
import spacy
from ast import literal_eval



### Constants

In [4]:
# connect with google drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
# change cwd
%cd drive/MyDrive/Work/Frontline/data

/content/drive/.shortcut-targets-by-id/1WfnZsqpG1r110J63sMbfS5TpsDOkveiV/data


### Data

In [15]:
from scripts import annotations

In [14]:
data=pd.read_csv("annotated/230621_all_annotationsV1.csv", index_col=0, converters={"annotations":literal_eval})

In [16]:
data.loc[:,"annotations"]=data.apply(annotations.ground_truth_filter,min_coannotation=1,min_similarity=-1, similarity="dice",axis=1)

In [ ]:
data.head()

One-hot-encoding of the target labels

In [19]:
data.annotations=data.annotations.apply(list)
data.annotations=["*".join(i) for i in data.annotations]

In [20]:
df=pd.merge(data[["text","artikel_id"]],data.annotations.str.get_dummies(sep="*"), left_index=True, right_index=True)

In [ ]:
# create a list of all labels
labels=df.columns.to_list()[-4:]

### Split data into train and test

In [25]:
#split train, dev , test sets
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42),
                            [int(.8 * len(df)), int(.9 * len(df))])

#### Convert DataFrame to  Dataset

In [26]:
ds=datasets.DatasetDict({"train":Dataset.from_dict(df_train),"val":Dataset.from_dict(df_val),"test":Dataset.from_dict(df_test)})

### Tokenizing & Encoding

In [27]:
tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-560m")

Downloading:   0%|          | 0.00/222 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.8M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

In [28]:
def preprocess_data(df):
  text = df["text"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)
  # add labels

  labels_batch = {k: df[k] for k in df.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]
  encoding["labels"] = labels_matrix.tolist()
  return encoding

In [29]:
ds_encoded=ds.map(preprocess_data, batched=True, remove_columns=ds['train'].column_names)

Map:   0%|          | 0/1547 [00:00<?, ? examples/s]

Map:   0%|          | 0/193 [00:00<?, ? examples/s]

Map:   0%|          | 0/194 [00:00<?, ? examples/s]

In [30]:
ds_encoded.set_format("torch")


### Modelling

In [31]:
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}

In [32]:
from transformers import AutoModelForSequenceClassification


In [33]:
model = AutoModelForSequenceClassification.from_pretrained("bigscience/bloom-560m",
                                   problem_type="multi_label_classification",
                                   num_labels=len(labels),
                                   id2label=id2label,
                                   label2id=label2id)

Downloading:   0%|          | 0.00/693 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

Some weights of BloomForSequenceClassification were not initialized from the model checkpoint at bigscience/bloom-560m and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Evaluate

In [34]:
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):

    preds = p.predictions[0] if isinstance(p.predictions,
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds,
        labels=p.label_ids)
    return result

In [35]:
ds_encoded["train"]=ds_encoded["train"].rename_column("labels","label")
ds_encoded["test"]=ds_encoded["test"].rename_column("labels","label")
ds_encoded["val"]=ds_encoded["val"].rename_column("labels","label")


In [36]:
args = TrainingArguments(
    f"bloom-finetuned-tagging-german",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=6,
    per_device_eval_batch_size=6,
    num_train_epochs=10,
    weight_decay=0.05,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    #push_to_hub=True,
)

In [37]:
trainer = Trainer(
    model,
    args,
    train_dataset=ds_encoded["train"],
    eval_dataset=ds_encoded["val"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

## Predicting

In [39]:
sigmoid = torch.nn.Sigmoid()

In [40]:
test_pred=pd.DataFrame(ds["test"])

In [41]:
def predict_text(text):
  encoding = tokenizer(text, return_tensors="pt")
  encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}
  outputs = trainer.model(**encoding)
  logits = outputs.logits
  probs = sigmoid(logits.squeeze().cpu())
  predictions = np.zeros(probs.shape)
  predictions[np.where(probs >= 0.5)] = 1
  # turn predicted id's into actual label names
  predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
  return predicted_labels


In [42]:
pred=[]
for t in ds["test"]["text"]:
  pred.append(predict_text(t))
test_pred["prediction"]=pred

In [43]:
test_pred.head()

,text,artikel_id,Domestic Violence,Graphic,Sensationalist,Statement of responsibility,prediction
0,""" In 30 Jahren haben sich auch die Themen der ...",14712573,1,0,0,0,[Domestic Violence]
1,Bei der Tat wurde laut dem Behördensprecher ei...,0E581BC770250A1CF5764784B2F7831C,1,0,0,0,[Domestic Violence]
2,Ennepe-Ruhr Mehr als 2000 Beratungsgespräche m...,150446aa-3fab-11eb-86d2-b93890c0e37b_1850238,1,0,0,0,[Domestic Violence]
3,Der 35-Jährige ist nicht mehr dringend tatverd...,157782084,1,0,0,0,[Domestic Violence]
4,Kreis Waldshut/Lauchringen - Mit einer neuen G...,1126537,1,0,0,0,[Domestic Violence]


### Calculate Performance
Calculate Hamming score as indication for predictions performance.

In [44]:
def hamming_score(y_true, y_pred):
    return (
        (y_true & y_pred).sum(axis=1) / (y_true | y_pred).sum(axis=1)
    ).mean()


In [45]:
predicted_labels=pd.DataFrame(pred)[0].str.get_dummies()
actual_labels=test_pred.iloc[:,1:4]

In [46]:
hamming_score(actual_labels, predicted_labels)

0.9226804123711341

Performance for Articles not labelled "Domestic Violence"

In [47]:
predicted_labels=predicted_labels[test_pred["Domestic Violence"]!=1]
actual_labels=actual_labels[test_pred["Domestic Violence"]!=1]

In [48]:
hamming_score(actual_labels, predicted_labels)

0.0